In [71]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font
from openpyxl.styles import Alignment
import time

name_file_data_report = 'Общая таблица слушателей ЦОПП 1-ПО.xlsx' 
path_to_end_folder_report = 'data'

df_po = pd.read_excel(name_file_data_report,sheet_name='ПО',dtype={'Гражданство_получателя_код_страны_по_ОКСМ':str})

#Создаем шрифт которым будем выделять названия таблиц
font_name_table = Font(name='Arial Black',size=15,italic=True)

C:\Users\1\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


1.3,
2.1.1,
2.1.2,
2.1.3,
2.2 Добавить категории слушателей для 1-ПО
2.3
2.4
2.5
2.6


In [72]:
# Создаем файл excel
wb = openpyxl.Workbook()
# Создаем листы 
wb.create_sheet(title='Раздел 2.5',index=0)
wb.create_sheet(title='Раздел 2.6',index=1)



<Worksheet "Раздел 2.6">

In [73]:
# Создаем дополнительную числовую колонку где каждое значение это 1, для удобства агрегирования
df_po_2_5 = df_po.copy()
# Добавляем колонку с 1
df_po_2_5['for_counting'] = 1 


In [74]:
# Считаем строку 01 Всего
df_2_5_all = pd.pivot_table(df_po_2_5,              
              columns=['Программа_профессионального_обучения_направление_подготовки','Источник_финансирования_обучения'],
              values=['for_counting'],
              aggfunc='sum')
df_2_5_all.index = ['Всего']
df_2_5_all.index.name='Код государства по ОКСМ'

In [75]:
df_2_5_all

Программа_профессионального_обучения_направление_подготовки Программа переподготовки рабочих, служащих  \
Источник_финансирования_обучения                                                        Местный бюджет   
Код государства по ОКСМ                                                                                  
Всего                                                                                                1   

Программа_профессионального_обучения_направление_подготовки                   \
Источник_финансирования_обучения                            Платное обучение   
Код государства по ОКСМ                                                        
Всего                                                                      1   

Программа_профессионального_обучения_направление_подготовки                      \
Источник_финансирования_обучения                            Региональный бюджет   
Код государства по ОКСМ                                                           
Всего                                                                         1   

Программа_профессионального_обучения_направление_подготовки                     \
Источник_финансирования_обучения                            Федеральный бюджет   
Код государства по ОКСМ                                                          
Всего                                                                        3   

Программа_профессионального_обучения_направление_подготовки Программа профессиональной подготовки по профессии рабочего, должности служащего   
Источник_финансирования_обучения                                                                                             Платное обучение  
Код государства по ОКСМ                                                                                                                        
Всего                                                                                                        4

In [76]:
# Считаем данные по странам
svod_df_po_2_5=pd.pivot_table(df_po_2_5,
              index=['Гражданство_получателя_код_страны_по_ОКСМ'],
              columns=['Программа_профессионального_обучения_направление_подготовки','Источник_финансирования_обучения'],
              values=['for_counting'],
              aggfunc='sum')

In [77]:
# Удаляем лишний мультииндекс
svod_df_po_2_5.columns= svod_df_po_2_5.columns.droplevel()
# заполняем нулями для корректного суммирования
svod_df_po_2_5.fillna(0.0,inplace=True)


In [78]:
# Соединяем датафреймы
df_2_5_out = pd.concat([df_2_5_all,svod_df_po_2_5])
# заменяем нули на нан, чтобы в итоговой таблице нули не отвлекали
df_2_5_out.replace(0.0,np.NaN,inplace=True)

In [79]:
df_2_5_out

Программа_профессионального_обучения_направление_подготовки Программа переподготовки рабочих, служащих  \
Источник_финансирования_обучения                                                        Местный бюджет   
Всего                                                                                              1.0   
004                                                                                                NaN   
012                                                                                                NaN   
020                                                                                                NaN   
643                                                                                                1.0   

Программа_профессионального_обучения_направление_подготовки                   \
Источник_финансирования_обучения                            Платное обучение   
Всего                                                                    1.0   
004                                                                      NaN   
012                                                                      NaN   
020                                                                      NaN   
643                                                                      1.0   

Программа_профессионального_обучения_направление_подготовки                      \
Источник_финансирования_обучения                            Региональный бюджет   
Всего                                                                       1.0   
004                                                                         NaN   
012                                                                         NaN   
020                                                                         NaN   
643                                                                         1.0   

Программа_профессионального_обучения_направление_подготовки                     \
Источник_финансирования_обучения                            Федеральный бюджет   
Всего                                                                      3.0   
004                                                                        NaN   
012                                                                        NaN   
020                                                                        NaN   
643                                                                        3.0   

Программа_профессионального_обучения_направление_подготовки Программа профессиональной подготовки по профессии рабочего, должности служащего   
Источник_финансирования_обучения                                                                                             Платное обучение  
Всего                                                                                                      4.0                                 
004                                                                                                        2.0                                 
012                                                                                                        1.0                                 
020                                                                                                        1.0                                 
643                                                                                                        NaN

In [80]:
wb['Раздел 2.5'][f'A1'] = 'Распределение слушателей, обученных по программам ПО, по гражданству'
wb['Раздел 2.5'][f'A1'].font = font_name_table
wb['Раздел 2.5'][f'A2'] = 'В таб'
wb['Раздел 2.5'][f'A2'].font = font_name_table



for r in dataframe_to_rows(df_2_5_out,index =True,header=True):
    if len(r) != 1:
        wb['Раздел 2.5'].append(r)
wb['Раздел 2.5'].column_dimensions['A'].width = 50
wb['Раздел 2.5'].column_dimensions['B'].width = 50
wb['Раздел 2.5'].column_dimensions['F'].width = 50

In [81]:
 # Получаем текущее время для того чтобы использовать в названии
t = time.localtime()
current_time = time.strftime('%H_%M_%S', t)
# Сохраняем итоговый файл
wb.save(f'{path_to_end_folder_report}/Часть отчета 1-ПО.xlsx {current_time}.xlsx')